# Anslysis using `polars`

I have followed **CampusX's - "SQL Case Study Course"** where **Ajay Sati Sir** perform analysis on a **Dataset - "Data Science Jobs Salaries in 2024"**.

> You can refer to [`data/README.md`](/data/README.md) to more about data.

In addition, to that session I have solved the same questions (given by sir in the session) using [`polars`](https://pola.rs) library.

> I have also provided the SQL solutions (solved by sir) from the session.

In [1]:
import polars as pl

In [2]:
pl.Config.set_fmt_str_lengths(40)

polars.config.Config

In [3]:
df = pl.read_csv("data/data_job_salaries.csv")
print(df.shape)
df.head()

(13972, 11)


work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
i64,str,str,str,i64,str,i64,str,i64,str,str
2024,"""SE""","""FT""","""AI Engineer""",90000,"""USD""",90000,"""AE""",0,"""AE""","""L"""
2024,"""SE""","""FT""","""Machine Learning Engineer""",180500,"""USD""",180500,"""US""",0,"""US""","""M"""
2024,"""SE""","""FT""","""Machine Learning Engineer""",96200,"""USD""",96200,"""US""",0,"""US""","""M"""
2024,"""SE""","""FT""","""Machine Learning Engineer""",235000,"""USD""",235000,"""AU""",0,"""AU""","""M"""
2024,"""SE""","""FT""","""Machine Learning Engineer""",175000,"""USD""",175000,"""AU""",0,"""AU""","""M"""


### 1. You're a compensation analyst employed by a multinational corporation. Your assignment is to pinpoint countries who give work fully remotely work , for the title `"Managers"` paying salaries exceeding **$90,000 USD**.

```sql
SELECT DISTINCT company_location
FROM            salaries
WHERE           job _title LIKE '%Manager%'
AND             salary_in_usd > 90000
AND             remote_ratio=100
```

In [13]:
(
    df.filter(
        pl.col("remote_ratio").eq(100),
        pl.col("job_title").str.contains(r"(?i)manager"),
        pl.col("salary_in_usd").ge(90_000),
    ).select(
        pl.col("company_location").unique(),
    )
)

company_location
str
"""FR"""
"""US"""
"""MX"""
"""AU"""


### 2. As a remote work advocate working for a progressive HR Tech Startup who place their freshers' clients in large tech firms. You're tasked with identifying top 5 country having greatest count of large (`company_size`) number of companies.

```sql
SELECT company_location,
       Count(*) AS 'cnt'
FROM   (SELECT *
        FROM   salaries
        WHERE  experience_level = 'EN'
               AND company_size = 'L')t
GROUP  BY company_location
ORDER  BY 'cnt' DESC
LIMIT  5
```

In [17]:
(
    df.filter(
        pl.col("experience_level").eq("EN"),
        pl.col("company_size").eq("L"),
    )
    .group_by("company_location")
    .len()
    .top_k(5, by="len")
)

company_location,len
str,u32
"""US""",53
"""DE""",10
"""CA""",10
"""GB""",8
"""IN""",6


### 3. Picture yourself as a data scientist working for a workforce management platform. Your objective is to calculate the percentage of employees. Who enjoy fully remote roles with salaries exceeding **$100,000 USD**, shedding light on the attractiveness of high-paying remote positions in today's job market.

```sql
SET @total = (SELECT Count(*) FROM salaries WHERE salary_in_usd>100000);

SET @count = (SELECT Count(*) FROM salaries WHERE salary_in_usd>100000 AND
remote_ratio=100);

SET @percentage= Round(((SELECT @count)/(SELECT @total))*100, 2);

SELECT @percentage AS 'Percentage' 
```

In [26]:
remote_employee_count = df.filter(
    pl.col("remote_ratio").eq(100),
    pl.col("salary_in_usd").ge(100_000),
).height
total_employee_count = df.filter(
    pl.col("salary_in_usd").ge(100_000),  # I don't think this is required
).height

"Percentage(%%) of employees enjoying remote roles with exceeding $100,000 USD salary is %.2f%%" % (
    (remote_employee_count / total_employee_count) * 100
)

'Percentage(%) of employees enjoying remote roles with exceeding $100,000 USD salary is 32.26%'

In [28]:
total_employee_count_wof = df.height  # Without filtering (wof)
"Percentage(%%) of employees enjoying remote roles with exceeding $100,000 USD salary is %.2f%%" % (
    (remote_employee_count / total_employee_count_wof) * 100
)

'Percentage(%) of employees enjoying remote roles with exceeding $100,000 USD salary is 25.00%'

### 4. Imagine you're a data analyst working for a global recruitment agency. Your task is to identify the locations where average salaries exceed the average salary for that job title in market for entry level, helping your agency guide candidates towards lucrative opportunities.

```sql
SELECT t.job_title,
       company_location,
       'average',
       'avg_per_country'
FROM   (SELECT job_title,
               Avg(salary_in_usd) AS 'average'
        FROM   salaries
        GROUP  BY job_title)t
       INNER JOIN (SELECT company_location,
                          job_title,
                          Avg(salary_in_usd) AS 'avg_per_country'
                   FROM   salaries
                   GROUP  BY job_title,
                             company_location)m
               ON t.job_title = m.job_title
WHERE  'avg_per_country' > 'average'
```

In [36]:
df.group_by("job_title").agg(
    pl.col("salary_in_usd").mean().name.suffix("_overall"),
).join(
    df.group_by("job_title", "company_location").agg(
        pl.col("salary_in_usd").mean().name.suffix("_country_wise"),
    ),
    on="job_title",
).filter(
    pl.col("salary_in_usd_country_wise").gt(pl.col("salary_in_usd_overall")),
)

job_title,salary_in_usd_overall,company_location,salary_in_usd_country_wise
str,f64,str,f64
"""Decision Scientist""",166094.631579,"""US""",167119.444444
"""Big Data Architect""",126751.0,"""GB""",153799.0
"""Machine Learning Scientist""",189348.184211,"""US""",195506.319149
"""Data Developer""",99383.136364,"""US""",101371.45
"""AI Developer""",135466.789474,"""CA""",275000.0
…,…,…,…
"""Computer Vision Software Engineer""",77760.6,"""AU""",150000.0
"""BI Data Analyst""",69026.529412,"""FR""",105066.0
"""AI Developer""",135466.789474,"""DE""",140358.5


### 5. You've been hired by a big HR Consultancy to look at how much people get paid in different countries. Your job is to find out for each job title which country pays the maximum average salary. This helps you to place your candidates in those countries.

```sql
SELECT *
FROM  (SELECT *,
              Dense_rank()
                OVER (
                  partition BY job_title
                  ORDER BY 'average' DESC) AS num
       FROM   (SELECT company_location,
                      job_title,
                      Avg(salary_in_usd) AS 'average'
               FROM   salaries
               GROUP  BY company_location,
                         job_title)t)k
WHERE  num = 1
```

In [15]:
(
    df.group_by("job_title", "company_location")
    .agg(
        pl.col("salary_in_usd").mean(),
    )
    .with_columns(
        pl.col("salary_in_usd").rank(descending=True).over("job_title").alias("rank"),
    )
    .filter(
        pl.col("rank").eq(1),
    )
)

job_title,company_location,salary_in_usd,rank
str,str,f64,f64
"""Principal Data Engineer""","""US""",192500.0,1.0
"""Data Operations Specialist""","""US""",87634.0,1.0
"""Head of Machine Learning""","""US""",337000.0,1.0
"""Computer Vision Engineer""","""US""",205278.26087,1.0
"""Data Operations Analyst""","""GB""",110500.0,1.0
…,…,…,…
"""Data Architect""","""US""",166972.970874,1.0
"""Data Strategist""","""US""",104345.909091,1.0
"""Data Science Tech Lead""","""US""",375000.0,1.0


In [12]:
(
    df.group_by("job_title", "company_location")
    .agg(
        pl.col("salary_in_usd").mean(),
    )
    .with_columns(
        pl.col("salary_in_usd").rank().over("job_title").alias("new"),
    )
    .sort("new", descending=True)
)

job_title,company_location,salary_in_usd,new
str,str,f64,f64
"""Data Scientist""","""NZ""",164951.5,39.0
"""Data Scientist""","""US""",162605.5684,38.0
"""Data Scientist""","""CA""",159513.139785,37.0
"""Data Scientist""","""EG""",144375.0,36.0
"""Data Scientist""","""CH""",124195.0,35.0
…,…,…,…
"""Data Integration Developer""","""US""",140580.75,1.0
"""Head of Data Science""","""IN""",60805.0,1.0
"""Lead Data Analyst""","""IN""",25821.666667,1.0


### 6. As a data-driven business consultant, you've been hired by a multinational corporation to analyze salary trends across different company locations. Your goal is to pinpoint locations where the average salary has consistently increased over the past few years. Countries where data is available for 3 years only (this and past two years) providing insights into locations experiencing sustained salary growth.

```sql
WITH t
     AS (SELECT *
         FROM   salaries
         WHERE  company_location IN (SELECT company_location
                                     FROM   (SELECT company_location,
                                                    Avg(salary_in_usd)        AS
                                                    AVG_salary,
                                                    Count(DISTINCT work_year) AS
                                                    num_years
                                             FROM   salaries
                                             WHERE  work_year >= Year(
                                                    CURRENT_DATE())
                                                                 - 2
                                             GROUP  BY company_location
                                             HAVING num_years = 3)m)) -- step 4
-- SELECT company_locatiON, work_year, AVG(salary_IN_usd) AS average FROM  t GROUP BY company_locatiON, work_year 
SELECT company_location,
       Max(CASE
             WHEN work_year = 2022 THEN average
           END) AS AVG_salary_2022,
       Max(CASE
             WHEN work_year = 2023 THEN average
           END) AS AVG_salary_2023,
       Max(CASE
             WHEN work_year = 2024 THEN average
           END) AS AVG_salary_2024
FROM   (SELECT company_location,
               work_year,
               Avg(salary_in_usd) AS average
        FROM   t
        GROUP  BY company_location,
                  work_year)q
GROUP  BY company_location
HAVING avg_salary_2024 > avg_salary_2023
       AND avg_salary_2023 > avg_salary_2022 -- step 3 and havINg step 4.

--------------------------------------
SELECT company_location,
       work_year,
       Avg(salary_in_usd) AS AVG_salary
FROM   salaries
GROUP  BY company_location,
          work_year -- step 1

SELECT company_location,
       work_year,
       Avg(salary_in_usd) AS AVG_salary
FROM   salaries
WHERE  work_year >= Year(CURRENT_DATE()) - 2
GROUP  BY company_location,
          work_year -- step 2

SELECT company_location,
       Avg(salary_in_usd)        AS AVG_salary,
       Count(DISTINCT work_year) AS num_years
FROM   salaries
WHERE  work_year >= Year(CURRENT_DATE()) - 2
GROUP  BY company_location
HAVING num_years = 3 -- STEP 3
```

In [8]:
from datetime import date

In [58]:
CURRENT_YEAR = date.today().year

In [59]:
# Here polars's LazyFrame is working as CTE (Commonn Table Expression) in SQL
contries_with_3_years = (
    df.lazy()
    .group_by("company_location")
    .agg(
        pl.col("salary_in_usd").mean(),
        pl.col("work_year").filter(pl.col("work_year").ge(CURRENT_YEAR - 2)).n_unique(),
    )
    .filter(
        pl.col("work_year").eq(3),
    )
    .select("company_location")
    # Now exract those companies which has 3 continous years (current and past 2 years)
    .join(df.lazy(), on="company_location", how="left")
    # When we join all the years came in the result, so filter it out
    .filter(
        pl.col("work_year").ge(CURRENT_YEAR - 2),
    )
    .select("company_location", "work_year", "salary_in_usd")
)

In [68]:
contries_with_3_years.collect().pivot(
    values="salary_in_usd",
    columns="work_year",
    index="company_location",
    aggregate_function="mean",
).filter(
    pl.col(str(CURRENT_YEAR)).ge(pl.col(str(CURRENT_YEAR - 1))),
    pl.col(str(CURRENT_YEAR - 1)).ge(pl.col(str(CURRENT_YEAR - 2))),
)

company_location,2024,2023,2022
str,f64,f64,f64
"""PT""",53054.75,51521.0,48921.375
"""IN""",71538.333333,47777.521739,37328.333333
"""FR""",101370.166667,100411.190476,72684.466667
"""AR""",88500.0,65000.0,50000.0
"""HU""",63333.0,43000.0,17684.0
"""ES""",72184.666667,60327.985714,47997.341463
"""FI""",77777.0,71259.0,63040.0
"""CA""",153611.807692,150724.141414,126009.552632


### 7. Picture yourself as a workforce strategist employed by a global HR tech startup. Your mission is to determine the percentage of  fully remote work for each experience level in 2021 and compare it with the corresponding figures for 2024, highlighting any significant increases or decreases in remote work adoption over the years.

```sql 
WITH t1
     AS (SELECT a.experience_level,
                total_remote,
                total_2021,
                Round(( ( ( total_remote ) / total_2021 ) * 100 ), 2) AS
                '2021 remote %'
         FROM   (SELECT experience_level,
                        Count(experience_level) AS total_remote
                 FROM   salaries
                 WHERE  work_year = 2021
                        AND remote_ratio = 100
                 GROUP  BY experience_level)a
                INNER JOIN (SELECT experience_level,
                                   Count(experience_level) AS total_2021
                            FROM   salaries
                            WHERE  work_year = 2021
                            GROUP  BY experience_level)b
                        ON a.experience_level = b.experience_level),
     t2
     AS (SELECT a.experience_level,
                total_remote,
                total_2024,
                Round(( ( ( total_remote ) / total_2024 ) * 100 ), 2)AS
                '2024 remote %'
         FROM   (SELECT experience_level,
                        Count(experience_level) AS total_remote
                 FROM   salaries
                 WHERE  work_year = 2024
                        AND remote_ratio = 100
                 GROUP  BY experience_level)a
                INNER JOIN (SELECT experience_level,
                                   Count(experience_level) AS total_2024
                            FROM   salaries
                            WHERE  work_year = 2024
                            GROUP  BY experience_level)b
                        ON a.experience_level = b.experience_level)
SELECT *
FROM   t1
       INNER JOIN t2
               ON t1.experience_level = t2.experience_level 
```

In [69]:
df.head()

work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
i64,str,str,str,i64,str,i64,str,i64,str,str
2024,"""SE""","""FT""","""AI Engineer""",90000,"""USD""",90000,"""AE""",0,"""AE""","""L"""
2024,"""SE""","""FT""","""Machine Learning Engineer""",180500,"""USD""",180500,"""US""",0,"""US""","""M"""
2024,"""SE""","""FT""","""Machine Learning Engineer""",96200,"""USD""",96200,"""US""",0,"""US""","""M"""
2024,"""SE""","""FT""","""Machine Learning Engineer""",235000,"""USD""",235000,"""AU""",0,"""AU""","""M"""
2024,"""SE""","""FT""","""Machine Learning Engineer""",175000,"""USD""",175000,"""AU""",0,"""AU""","""M"""


In [81]:
# Not The Answer
# Comparision of mean `remote_ratio` of each years
(
    df
    .group_by("experience_level", "work_year")
    .agg(
        pl.col("remote_ratio").mean().round(2),
    )
    .pivot(
        values="remote_ratio",
        index="experience_level",
        columns="work_year",
    )
    .select("experience_level", pl.col(map(str, range(2021, 2025))))
)

experience_level,2021,2022,2023,2024
str,f64,f64,f64,f64
"""EN""",69.57,59.91,32.61,23.62
"""MI""",68.97,53.21,28.35,20.87
"""SE""",72.67,54.79,32.48,25.18
"""EX""",60.0,71.95,36.78,33.49


In [20]:
df["experience_level"].value_counts()

experience_level,count
str,u32
"""MI""",3294
"""EX""",403
"""SE""",9248
"""EN""",1027


In [21]:
df["work_year"].value_counts()

work_year,count
i64,u32
2021,218
2023,8518
2024,3509
2022,1652
2020,75


In [30]:
remote_pct_2021 = (
    df.lazy()
    .filter(
        pl.col("work_year").eq(2021),
        pl.col("remote_ratio").eq(100),
    )
    .group_by("experience_level")
    .agg(
        pl.len().alias("remote_emp_2021"),
    )
    .join(
        (
            df.lazy()
            .filter(
                pl.col("work_year").eq(2021),
            )
            .group_by("experience_level")
            .agg(pl.len().alias("total_emp"))
        ),
        on="experience_level",
    )
    .with_columns(
        pl.col("remote_emp_2021")
        .truediv(pl.col("total_emp"))
        .mul(100)
        .round(2)
        .alias("remote_pct_2021"),
    )
)

remote_pct_2024 = (
    df.lazy()
    .filter(
        pl.col("work_year").eq(2024),
        pl.col("remote_ratio").eq(100),
    )
    .group_by("experience_level")
    .agg(
        pl.len().alias("remote_emp_2024"),
    )
    .join(
        (
            df.lazy()
            .filter(
                pl.col("work_year").eq(2024),
            )
            .group_by("experience_level")
            .agg(pl.len().alias("total_emp"))
        ),
        on="experience_level",
    )
    .with_columns(
        pl.col("remote_emp_2024")
        .truediv(pl.col("total_emp"))
        .mul(100)
        .round(2)
        .alias("remote_pct_2024"),
    )
)

remote_pct_2021.select("experience_level", "remote_pct_2021").join(
    remote_pct_2024.select("experience_level", "remote_pct_2024"), on="experience_level"
).collect()

experience_level,remote_pct_2021,remote_pct_2024
str,f64,f64
"""EN""",47.83,22.83
"""MI""",51.72,20.6
"""EX""",50.0,33.02
"""SE""",58.67,25.16


### 8. As a compensation specialist at a fortune 500 company, you're tasked with analyzing salary trends over time. Your objective is to calculate the average salary increase percentage for each experience level and job title between the years 2023 and 2024, helping the company stay competitive in the talent market.

```sql  
WITH t
     AS (SELECT experience_level,
                job_title,
                work_year,
                Round(Avg(salary_in_usd), 2) AS 'average'
         FROM   salaries
         WHERE  work_year IN ( 2023, 2024 )
         GROUP  BY experience_level,
                   job_title,
                   work_year) -- step 1
SELECT *,
       Round(( ( ( avg_salary_2024 - avg_salary_2023 ) / avg_salary_2023 ) * 100
             ), 2)
       AS changes
FROM   (SELECT experience_level,
               job_title,
               Max(CASE
                     WHEN work_year = 2023 THEN average
                   END) AS AVG_salary_2023,
               Max(CASE
                     WHEN work_year = 2024 THEN average
                   END) AS AVG_salary_2024
        FROM   t
        GROUP  BY experience_level,
                  job_title -- step 2
       )a
WHERE  ( ( ( avg_salary_2024 - avg_salary_2023 ) / avg_salary_2023 ) * 100 ) IS
       NOT
       NULL -- STEP 3
```

### 9. You're a database administrator tasked with role-based access control for a company's employee database. Your goal is to implement a security measure where employees in different experience level (e.g.Entry Level, Senior level etc.) can only access details relevant to their respective experience_level, ensuring data confidentiality and minimizing the risk of unauthorized access.

```sql  
SELECT DISTINCT experience_level FROM salaries;

CREATE USER 'Entry_level'@'%' IDENTIFIED BY 'EN';
CREATE USER 'Junior_Mid_level'@'%' IDENTIFIED BY ' MI '; 
CREATE USER 'Intermediate_Senior_level'@'%' IDENTIFIED BY 'SE';
CREATE USER 'Expert Executive-level '@'%' IDENTIFIED BY 'EX ';

CREATE VIEW entry_level AS
SELECT * FROM salaries WHERE experience_level='EN'

GRANT SELECT ON campusx.entry_level TO 'Entry_level'@'%'

UPDATE view entry_level SET WORK_YEAR = 2025 WHERE EMPLOYNMENT_TYPE='FT'
```

### 10. You are working with an consultancy firm, your client comes to you with certain data and preferences such as ( their year of experience , their employment type, company location and company size )  and want to make an transaction into different domain in data industry (like  a person is working as a data analyst and want to move to some other domain such as data science or data engineering etc.) Your work is to  guide them to which domain they should switch to base on  the input they provided, so that they can now update thier knowledge as per the suggestion. The Suggestion should be based on average salary.

```sql  
DELIMITER //
CREATE PROCEDURE GetAverageSalary(IN exp_lev VARCHAR(2), IN emp_type VARCHAR(3), IN comp_loc VARCHAR(2), IN comp_size VARCHAR(2))
BEGIN
    SELECT job_title, experience_level, company_location, company_size, employment_type, ROUND(AVG(salary), 2) AS avg_salary 
    FROM salaries 
    WHERE experience_level = exp_lev AND company_location = comp_loc AND company_size = comp_size AND employment_type = emp_type 
    GROUP BY experience_level, employment_type, company_location, company_size, job_title ORDER BY avg_salary DESC ;
END//
DELIMITER ;
-- Deliminator  By doing this, you're telling MySQL that statements within the block should be parsed as a single unit until the custom delimiter is encountered.

CALL GetAverageSalary('EN','FT','AU','M')
DROP PROCEDURE Getaveragesalary
```